In [1]:
import pandas as pd

In [2]:
import seaborn as sns

In [3]:
import matplotlib.pyplot as plt

In [4]:
from sklearn.feature_extraction import DictVectorizer
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Lasso
from sklearn.linear_model import Ridge

from sklearn.metrics import mean_squared_error

In [5]:


def read_dataframe(filename):
    if filename.endswith('.csv'):
        df = pd.read_csv(filename)

        df.tpep_dropoff_datetime = pd.to_datetime(df.tpep_dropoff_datetime)
        df.tpep_pickup_datetime = pd.to_datetime(df.tpep_pickup_datetime)
    elif filename.endswith('.parquet'):
        df = pd.read_parquet(filename)
        
    print(f'Column number {df.shape[1]}')

    df['duration'] = df.tpep_dropoff_datetime - df.tpep_pickup_datetime
    df.duration = df.duration.apply(lambda td: td.total_seconds() / 60)
    
    raw_count = len(df)
    
    print(df.std())

    df = df[(df.duration >= 1) & (df.duration <= 60)]
    
    filter_count = len(df)
    
    print(f'fraction or records with duration between 1 and 60 :{filter_count/raw_count}')

    categorical = ['PULocationID', 'DOLocationID']
    df[categorical] = df[categorical].astype(str)
    
    return df



In [6]:
filename = '../yellow_tripdata_2022-01.parquet'

In [7]:
df_train = read_dataframe(filename)

Column number 19


/tmp/ipykernel_163/3839496861.py:17: FutureWarning: The default value of numeric_only in DataFrame.std is deprecated. In a future version, it will default to False. In addition, specifying 'numeric_only=None' is deprecated. Select only valid columns or specify the value of numeric_only to silence this warning.
  print(df.std())


VendorID                                   0.502137
tpep_pickup_datetime     14 days 07:12:05.644016136
tpep_dropoff_datetime    14 days 07:11:17.325123812
passenger_count                            0.982969
trip_distance                            547.871404
RatecodeID                                 5.917573
PULocationID                              65.468057
DOLocationID                              70.790159
payment_type                               0.500178
fare_amount                              255.814887
extra                                      1.236226
mta_tax                                    0.082916
tip_amount                                 2.830698
tolls_amount                               1.680538
improvement_surcharge                      0.043747
total_amount                             255.964089
congestion_surcharge                       0.743204
airport_fee                                0.312555
duration                                  46.445305
dtype: objec

In [8]:
df_val = read_dataframe('../yellow_tripdata_2022-02.parquet')

Column number 19


/tmp/ipykernel_163/3839496861.py:17: FutureWarning: The default value of numeric_only in DataFrame.std is deprecated. In a future version, it will default to False. In addition, specifying 'numeric_only=None' is deprecated. Select only valid columns or specify the value of numeric_only to silence this warning.
  print(df.std())


VendorID                                   0.501401
tpep_pickup_datetime     12 days 03:31:10.275087219
tpep_dropoff_datetime    12 days 03:30:07.149904354
passenger_count                            0.984123
trip_distance                            642.530368
RatecodeID                                 6.017545
PULocationID                              65.609659
DOLocationID                              70.420229
payment_type                               0.495248
fare_amount                               12.046193
extra                                      1.234659
mta_tax                                    0.081491
tip_amount                                 2.788297
tolls_amount                               1.671188
improvement_surcharge                       0.04363
total_amount                              14.945992
congestion_surcharge                       0.709771
airport_fee                                 0.29637
duration                                  47.263941
dtype: objec

In [9]:
len(df_train), len(df_val)

(2421440, 2918187)

In [10]:
df_train['PU_DO'] = df_train['PULocationID'] + '_' + df_train['DOLocationID']
df_val['PU_DO'] = df_val['PULocationID'] + '_' + df_val['DOLocationID']

In [33]:
categorical = ['PULocationID', 'DOLocationID'] #'PULocationID', 'DOLocationID']
numerical = ['duration']

dv = DictVectorizer()

train_dicts = df_train[categorical].to_dict(orient='records')
X_train = dv.fit_transform(train_dicts)

print(len(dv.feature_names_))

val_dicts = df_val[categorical].to_dict(orient='records')
X_val = dv.transform(val_dicts)

515


In [36]:
target = 'duration'
y_train = df_train[target].values
y_val = df_val[target].values

In [37]:
lr = LinearRegression()
lr.fit(X_train, y_train)

y_pred = lr.predict(X_train)

mean_squared_error(y_train, y_pred, squared=False)

6.986190837370544

In [39]:
y_pred = lr.predict(X_val)

mean_squared_error(y_val, y_pred, squared=False)

7.786409085078911